In [1]:
import numpy as np
import matplotlib                                                                                                                   
matplotlib.use('Agg')   
import matplotlib.pyplot as plt
import pandas as pd

import os

from MSRF7_monotarget_fun import RF_train
from MSnn7_monotarget_fun import NN_train
from MSXB7_fun2 import XGBoost_train

print(__doc__)
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

import sklearn.preprocessing as skpp

Automatically created module for IPython interactive environment


In [3]:
def find_all(name, path):
    result = []
    for file in os.listdir(path):
        if file.startswith(name):
            result.append(file)
    return result

name='New_Matched-Rockstar-G3X-snap_' #name to search by
#name='New_Matched-Rockstar-G3X-snap_all'
#path=os.getcwd() #get current file dir
path= "/home2/frobledo/datafiles/V7/" #get file dir

new_columns=['num_prog(4)', 'sam_mvir(9)', 'mvir(10)', 'rvir(11)', 'rs(12)',\
       'vrms(13)', 'scale_of_last_MM(15)', 'vmax(16)', 'Spin(26)',\
       'Rs_Klypin(37)', 'Mmvir_all(38)', 'M200b(39)', 'M200c(40)', 'M500c(41)',\
       'M2500c(42)', 'Xoff(43)', 'Voff(44)', 'Spin_Bullock(45)', 'b_to_a(46)',\
       'c_to_a(47)', 'b_to_a(500c)(51)', 'c_to_a(500c)(52)', 'T/|U|(56)',\
       'Macc(59)', 'Mpeak(60)', 'Vacc(61)', 'Vpeak(62)', 'Halfmass_Scale(63)',\
       'First_Acc_Mvir(72)', 'First_Acc_Vmax(73)', 'Vmax\@Mpeak(74)','a',\
       'G3XMgas(80)','G3XMstar(81)' ,'G3XTgas_mw(82)', 'G3XYx(84)', 'G3XYsz(85)']


targetname=['G3XMgas(80)','G3XMstar(81)' ,'G3XTgas_mw(82)', 'G3XYx(84)', 'G3XYsz(85)']
monotarget=['G3XMgas(80)']

### Entrenamos los tres algoritmos en monoobjetivo

In [4]:
train_file_names=sorted(find_all(name,path))
setnames=[]
dfp_list=[]
for i in range(len(train_file_names)):
    setnames.append(str(train_file_names[i][:-8]))
    dfp_temp=pd.read_csv(path + train_file_names[i] , sep=',')
    
    dfp_temp= dfp_temp[new_columns]
    
    dfp_list.append( dfp_temp )
    

dfp_final=pd.concat(dfp_list)

    
    
#RF_train(dfp_final,targetname, setname='New_Matched_Rockstar-G3X-snap_merge_data')
print('RF all data done')

#for target in targetname:
#    XGBoost_train(dfp_final,target, setname='New_Matched_Rockstar-G3X-snap_merge_data')
    
#    print('XGBoost ' + target + ' in all data done')
#print('XGBoost all data done')

RF all data done


In [26]:
dfp=dfp_final
setname='New_Matched_Rockstar-G3X-snap_merge_data'
targetname=['G3XMgas(80)','G3XMstar(81)' ,'G3XTgas_mw(82)', 'G3XYx(84)', 'G3XYsz(85)']


print(setname + ' column list:')
print(dfp.columns)
print('-----------------------------------')

print('Data imported')
    

dfp=dfp[dfp['M500c(41)'] >13.5]

dfp=dfp.dropna(axis=1)
    
new_col_list=dfp.columns
    
print('New column list:')
print(new_col_list)
    
#    targetname=['G3XMgas(80)','G3XMstar(81)' ,'G3XTgas_mw(82)', 'G3XYx(84)', 'G3XYsz(85)']
    
dfp.reset_index(drop=True, inplace=True)

Y_train=dfp[targetname]
    

dfptrain=dfp.copy()
    
    
dfptrain.drop(labels=targetname, inplace=True, axis=1)
    
coldrop=[col for col in dfptrain.columns if 'G3X' in col]
dfptrain.drop(labels=coldrop, inplace=True, axis=1)
    
new_col_list=dfptrain.columns
print(new_col_list)       
#plot hist
plt.figure('Mass hist')
plt.hist(dfp['M500c(41)'], bins=20, zorder=1, label=[setname + ' set'])
plt.legend()
    
    
dfptrain[targetname]=Y_train
    
corr= dfptrain.corr()
    
plt.figure('Correlation matrix - training data', figsize=(9,9))
nticks=len(dfptrain.columns)
plt.xticks(range(nticks), [lab[0:7] for lab in dfptrain.columns], rotation='vertical')
plt.yticks(range(nticks), dfptrain.columns)
_ = plt.colorbar(plt.imshow(corr, interpolation='nearest', vmin=-1., vmax=1., cmap=plt.get_cmap('YlOrBr')))
plt.title('Correlation matrix - Training data', fontsize=20)
plt.savefig('plots/correlation/correlationFULL_plot.png')

New_Matched_Rockstar-G3X-snap_merge_data column list:
Index(['num_prog(4)', 'sam_mvir(9)', 'mvir(10)', 'rvir(11)', 'rs(12)',
       'vrms(13)', 'scale_of_last_MM(15)', 'vmax(16)', 'Spin(26)',
       'Rs_Klypin(37)', 'Mmvir_all(38)', 'M200b(39)', 'M200c(40)', 'M500c(41)',
       'M2500c(42)', 'Xoff(43)', 'Voff(44)', 'Spin_Bullock(45)', 'b_to_a(46)',
       'c_to_a(47)', 'b_to_a(500c)(51)', 'c_to_a(500c)(52)', 'T/|U|(56)',
       'Macc(59)', 'Mpeak(60)', 'Vacc(61)', 'Vpeak(62)', 'Halfmass_Scale(63)',
       'First_Acc_Mvir(72)', 'First_Acc_Vmax(73)', 'Vmax\@Mpeak(74)', 'a',
       'G3XMgas(80)', 'G3XMstar(81)', 'G3XTgas_mw(82)', 'G3XYx(84)',
       'G3XYsz(85)'],
      dtype='object')
-----------------------------------
Data imported
New column list:
Index(['num_prog(4)', 'sam_mvir(9)', 'mvir(10)', 'rvir(11)', 'rs(12)',
       'vrms(13)', 'scale_of_last_MM(15)', 'vmax(16)', 'Spin(26)',
       'Rs_Klypin(37)', 'Mmvir_all(38)', 'M200b(39)', 'M200c(40)', 'M500c(41)',
       'M2500c(42)', 'X

### Representamos gráficamente y guardamos

In [7]:
rf_feature=pd.read_csv('Feature importance for RF[G3XMgas(80)].csv')
#nn_feature=pd.read_csv('Feature importance for NN[G3XYsz(85)].csv')
#xgb_feature=pd.read_csv('datafiles/feature_importance_XB/Feature importance for G3XYsz(85).csv')

feature=rf_feature
#feature=nn_feature
#feature=xgb_feature

test=np.split(dfp_final2,[32],axis=1) #Tambien se podría simplemente quitar la última
X_test=test[0]
y_test=test[1]
columns=X_test.columns.values

#result = permutation_importance(clf, X_test, y_test, scoring='neg_mean_squared_error', n_repeats=10, random_state=42)
#perm_sorted_idx = result.importances_mean.argsort()


#Representaciones de permutation importance y feature importamce

importance_sorted_idx = np.argsort(feature.Weight)
importance_sorted_idx2 = np.argsort(feature.Paper)

indices = np.arange(0, len(feature.Weight)) + 0.5
indices2 = np.arange(0, len(feature.Paper)) + 0.5

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12, 8))

ax1.errorbar(feature.Weight[importance_sorted_idx],indices,xerr=feature.Error[importance_sorted_idx],
            fmt=".", ecolor='r',elinewidth=1,capsize=4)
ax1.set_yticklabels(columns[importance_sorted_idx])
ax1.set_yticks(indices)
ax1.set_ylim((0, len(feature.Weight)))

ax2.barh(indices2,
         feature.Paper[importance_sorted_idx2], height=0.7)
ax2.set_yticklabels(columns[importance_sorted_idx2])
ax2.set_yticks(indices2)
ax2.set_ylim((0, len(feature.Paper)))

#ax1.set_title('Feature importance Neural Networks for'+str(y_test.columns.values))
plt.xlabel('Error relativo de R^2 (%)')
ax1.set_title('Feature importance RF for' + str(y_test.columns.values) )
#ax1.set_title('Feature importance XGBoost for G3XYsz(85)')

#ax2.boxplot(result.importances[perm_sorted_idx].T, vert=False,
#           labels=columns[perm_sorted_idx])

fig.tight_layout()
plt.show()

plt.savefig('plots/RFFeatureImportance for'+str(y_test.columns.values)+'_plot.png') #RF
#plt.savefig('plots/NNFeatureImportance for' + str(y_test.columns.values) + '_plot.png') #NN
#plt.savefig('plots/XGBFeatureImportance for G3XYsz(85)_plot.png') 

## Con dimensionalidad reducida

In [30]:
print('CALCULO CON DIMENSIONALIDAD REDUCIDA')
from MSRF7_monotarget_fun import RF_train

dfp_dimred=dfp_final.drop(dfp_final.columns[[1,2,3,5,7,9,10,11,12,14,15,16,17,18,19,21,23,24,25,27,28,29,30]],axis=1)
           
#Re-entrenamos con las dimensiones reducidas en monoobjetivo

dfp_final2=dfp_dimred.drop(dfp_dimred.columns[[13,10,11,12]],axis=1)

targetname=['G3XMgas(80)','G3XMstar(81)' ,'G3XTgas_mw(82)', 'G3XYx(84)', 'G3XYsz(85)']
monotarget=['G3XMgas(80)']

#NN_train(dfp_final2, monotarget, setname='New_Matched_Rockstar-G3X-snap_merge_data_Reduced_Dim')
#print('NN all data done')
    

RF_train(dfp_final2,monotarget, setname='New_Matched_Rockstar-G3X-snap_merge_data_Reduced_Dim')
print('RF all data done')


CALCULO CON DIMENSIONALIDAD REDUCIDA
New_Matched_Rockstar-G3X-snap_merge_data_Reduced_Dim column list:
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a',
       'G3XMgas(80)'],
      dtype='object')
-----------------------------------
Data imported
New column list:
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a',
       'G3XMgas(80)'],
      dtype='object')
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
resizing Y


K-fold: 1
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
(9,)
y_min:
G3XMgas(80)    11.998705
dtype: float64
y_max:
G3XMgas(80)    14.393194
dtype: float64
Scaling done
Set

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/home2/innigo/MSRF7_monotarget_fun.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,Y_train)


(9,)
(9,)
Score training =  0.9980664431881685
Score test =  0.9891343841412421
MSE =  0.0003990340015102584
MAE =  0.013078644115843308
R^2 score = 0.9891343841412421


K-fold: 2
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
(9,)
y_min:
G3XMgas(80)    11.998705
dtype: float64
y_max:
G3XMgas(80)    14.393194
dtype: float64
Scaling done
Sets ready


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/home2/innigo/MSRF7_monotarget_fun.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,Y_train)


(9,)
(9,)
Score training =  0.9980154788672606
Score test =  0.9891294100860539
MSE =  0.00040586584235630547
MAE =  0.013215637255242812
R^2 score = 0.9891294100860539


K-fold: 3
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
(9,)
y_min:
G3XMgas(80)    11.998705
dtype: float64
y_max:
G3XMgas(80)    14.393194
dtype: float64
Scaling done
Sets ready


...

(9,)
(9,)
Score training =  0.9981043521953181
Score test =  0.9875502707780859
MSE =  0.0004502527887997802
MAE =  0.013391916414679439
R^2 score = 0.9875502707780859


K-fold: 4
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
(9,)
y_min:
G3XMgas(80)    11.998705
dtype: float64
y_max:
G3XMgas(80)    14.393194
dtype: float64
Scaling done
Sets ready


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/home2/innigo/MSRF7_monotarget_fun.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,Y_train)


(9,)
(9,)
Score training =  0.9980970539249931
Score test =  0.9885112515839533
MSE =  0.00041748515815193745
MAE =  0.01342643301296197
R^2 score = 0.9885112515839533


K-fold: 5
Index(['num_prog(4)', 'rs(12)', 'scale_of_last_MM(15)', 'Spin(26)',
       'M500c(41)', 'b_to_a(500c)(51)', 'T/|U|(56)', 'Vpeak(62)', 'a'],
      dtype='object')
(9,)
y_min:
G3XMgas(80)    11.998705
dtype: float64
y_max:
G3XMgas(80)    14.393194
dtype: float64
Scaling done
Sets ready


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
/home2/innigo/MSRF7_monotarget_fun.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,Y_train)


(9,)
(9,)
Score training =  0.9980490119498382
Score test =  0.9888061382350889
MSE =  0.00040818715251943774
MAE =  0.01293180208364184
R^2 score = 0.9888061382350889


MSE - 5 Folds :  0.0004161649886675439
MAE - 5 Folds :  0.013208886576473872
R^2 - 5 Folds :  0.9886262909648847
[1.05896455e+00 2.19648282e-01 2.49134229e-01 5.49353840e-02
 6.16791162e+01 4.48275453e-02 1.12522919e-01 1.75812965e+01
 7.14924346e-02]
Feature importance exported
Data exported
RF all data done


In [ ]:
dfp_dimred=dfp_final.drop(dfp_final.columns[[1,2,3,5,7,9,10,11,12,14,15,16,17,18,19,21,23,24,25,27,28,29,30]],axis=1)
           
#Re-entrenamos con las dimensiones reducidas en monoobjetivo

dfp_final2=dfp_dimred.drop(dfp_dimred.columns[[13,9,11,12]],axis=1)

## Gráfica triple 

In [22]:
#El nombre del archivo se modifica pà
feature2=pd.read_csv('datafiles/feature_importance_RF/NEWFeature importance for RF[G3XTgas_mw(82)]_ReducedDim.csv')
feature=pd.read_csv('Feature importance for RF[G3XTgas_mw(82)]_ReducedDim.csv')



#Pasamos a que sea el error relativo de verdad a mano (hay que modificarlo a mano para cada target)
feature2.Score=abs((feature2.Score-0.9828978514938731)*100/0.9828978514938731)
feature2.Error=abs((feature2.Error*100)/0.9828978514938731)

test=np.split(dfp_final2,[9],axis=1) #Tambien se podría simplemente quitar la última
X_test=test[0]
y_test2=test[1]
columns=X_test.columns.values

#result = permutation_importance(clf, X_test, y_test, scoring='neg_mean_squared_error', n_repeats=10, random_state=42)
#perm_sorted_idx = result.importances_mean.argsort()


#Representaciones de permutation importance y feature importamce

importance_sorted_idx = np.argsort(feature.Weight)
importance_sorted_idx2 = np.argsort(feature.Paper)

indices = np.arange(0, len(feature.Weight)) + 0.5
indices2 = np.arange(0, len(feature.Paper)) + 0.5

#Luego removerr
feature.Weight=feature.Weight/100
feature.Error=feature.Error/100

fig, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(15, 8))

ax1.errorbar(feature.Weight[importance_sorted_idx],indices,xerr=feature.Error[importance_sorted_idx],
            fmt=".", ecolor='r',elinewidth=1,capsize=4)
ax1.set_yticklabels(columns[importance_sorted_idx])
ax1.set_yticks(indices)
ax1.set_ylim((0, len(feature.Weight)))
ax1.set_xlabel('Error relativo de R^2 (%)')

ax2.barh(indices2,
         feature.Paper[importance_sorted_idx2], height=0.7)
ax2.set_yticklabels(columns[importance_sorted_idx2])
ax2.set_yticks(indices2)
ax2.set_ylim((0, len(feature.Paper)))
ax2.set_xlabel('Fracción relativa al uso en RF')

importance_sorted_idx3 = np.argsort(feature2.Score)
indices3 = np.arange(0, len(feature2.Score)) + 0.5

ax3.errorbar(feature2.Score[importance_sorted_idx3],indices3,xerr=feature2.Error[importance_sorted_idx3],
            fmt=".", ecolor='r',elinewidth=1,capsize=4)
ax3.set_yticklabels(columns[importance_sorted_idx3])
ax3.set_yticks(indices3)
ax3.set_ylim((0, len(feature2.Score)))

p='G3XTgas_mw(82)'
#ax1.set_title('Feature importance Neural Networks for'+str(y_test.columns.values))
plt.xlabel('Error relativo de R^2 (%)')
ax2.set_title('Feature importance RF for' +  str(p))
#ax1.set_title('Feature importance XGBoost for G3XYsz(85)')

#ax2.boxplot(result.importances[perm_sorted_idx].T, vert=False,
#           labels=columns[perm_sorted_idx])

fig.tight_layout()
plt.show()

plt.savefig('plots/RFFeatureImportance for[G3XTgas_mw(82)]ReducedDim_plot.png') #RF


## Representación del paper de la evolcuionde R2 y MSE  

In [6]:
eature=pd.read_csv(' xxxxxxx.csv') #Archivo conteniendo el R2 y MSE evolution



test=np.split(dfp_final2,[9],axis=1) #Tambien se podría simplemente quitar la última
X_test=test[0]
y_test2=test[1]
columns=X_test.columns.values


#indices = np.arange(0, len(feature.Weight)) + 0.5
#indices2 = np.arange(0, len(feature.Paper)) + 0.5

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(13, 6))

ax1.errorbar(feature.Name,feature.R2,yerr=feature.ErrorR2,
            fmt=".--", ecolor='r',elinewidth=1,linewidth=0.5,capsize=4)
ax1.set_xticklabels(feature.Name,rotation=50,ha='right')
#ax1.set_yticks(indices)
#ax1.set_ylim((0, len(feature.R2)))
ax1.set(ylabel='Evolución de R^2')

#importance_sorted_idx3 = np.argsort(feature2.Score)
#indices3 = np.arange(0, len(feature2.Score)) + 0.5

ax2.errorbar(feature.Name,feature.MSE,yerr=feature.ErrorMSE,
            fmt=".--", ecolor='r',elinewidth=1,linewidth=0.5,capsize=4)

ax2.set_xticklabels(feature.Name,rotation=50,ha='right')

ax2.set(ylabel='Evolución de MSE')
#ax2.set_title

fig.tight_layout()
plt.show()

plt.savefig('plots/RFEvolutionOfScore.png') #RF